In [1]:
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
# read data
random_ema = pd.read_csv('random-ema.csv')
puff_marker = pd.read_csv('puff-probability.csv')

random_ema_alternative = pd.read_csv('random-ema-alternative.csv')
puff_marker_alternative = pd.read_csv('puff-probability-alternative.csv')

random_ema_backup = pd.read_csv('random-ema-backup.csv')
puff_marker_backup = pd.read_csv('puff-probability-backup.csv')

In [ ]:
def random_puffmarker(random_ema, puff_marker):
    ema_stats = {}
    ema_times = {}
    puff_times = {}
    
    for index, row in puff_marker.iterrows():
        user = row['participant_id']
        if user not in puff_times:
            puff_times[user] = set()
        try:
            time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
        except ValueError:
            time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
        except TypeError:
            continue
        date = (time.year, time.month, time.day, int(row['hour']))
        puff_times[user].add(date)
    
    